In [2]:
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
import numpy as np

Sanitizando dados do Coconut

In [3]:
%%time

suppl = Chem.SDMolSupplier('coconut_05_2022.sdf')

coco = []

for mol in suppl:
    if type(mol) == Chem.rdchem.Mol:
        coco.append(mol.GetPropsAsDict())

[10:44:11] Explicit valence for atom # 20 C, 6, is greater than permitted
RDKit ERROR: [10:44:11] Explicit valence for atom # 20 C, 6, is greater than permitted
RDKit ERROR: [10:44:11] ERROR: Could not sanitize molecule ending on line 53
[10:44:11] ERROR: Could not sanitize molecule ending on line 53
RDKit ERROR: [10:44:11] ERROR: Explicit valence for atom # 20 C, 6, is greater than permitted
[10:44:11] ERROR: Explicit valence for atom # 20 C, 6, is greater than permitted
[10:44:14] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [10:44:14] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [10:44:14] ERROR: Could not sanitize molecule ending on line 329955
[10:44:14] ERROR: Could not sanitize molecule ending on line 329955


CPU times: user 3.98 s, sys: 24 ms, total: 4 s
Wall time: 4.11 s


In [4]:
coconut = pd.DataFrame(coco)

Carregando datasets de antibióticos e comparando com o Coconut, usando InChIKey

In [4]:
antibio = pd.read_csv('/home/joao/documentos/IC-Joao/classificador/antibio.csv', sep=';', encoding='latin-1')

In [5]:
len(set(antibio['InChIKey']).intersection(set(coconut['inchikey'])))

0

In [6]:
antibio2 = pd.read_excel('/home/joao/documentos/IC-Joao/classificador/antibio2.xlsx')

In [7]:
antb2 = []

for x in antibio2['INCHI']:
    try:
        antb2.append(Chem.InchiToInchiKey(x))
    except:
        antb2.append('')


antibio2['InChIKey'] = antb2
antibio2 = antibio2[antibio2['InChIKey']!='']

In [8]:
len((set(coconut['inchikey'])).intersection(set(antibio2['InChIKey'])))

0

In [9]:
len((set(antibio2['InChIKey'])).intersection(set(antibio['InChIKey'])))


12

Juntando os dois conjuntos de dados

In [10]:
antibio2.columns

Index(['Name', 'SMILES', 'Molecular Formula', 'LogP', 'Molecular Weight',
       'HBD', 'HBA', 'TPSA', 'ROTB', 'AROM', 'ALERTS', 'INCHI', 'InChIKey'],
      dtype='object')

In [11]:
antibio2a = antibio2[['Name','SMILES','InChIKey']]

In [13]:
antibioa = antibio[['Name','SMILES','InChIKey']]

In [14]:
frames = [antibioa,antibio2a]
antibiotics_list = pd.concat(frames)

In [15]:
# Baixei a combinação para excluir as duplicatas manualmente
# from pathlib import Path  
# filepath = Path('/home/joao/documentos/IC-Joao/classificador/antibioticlist.xlsx')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# antibiotics_list.to_excel(filepath,index=False) 

In [16]:
#Criei uma variável com as duplicadas e seus indices para saber qual remover
dup = antibiotics_list['InChIKey'].duplicated()

In [17]:
# Criei um csv para poder visualizar todos os índices
# from pathlib import Path  
# filepath = Path('/home/joao/documentos/IC-Joao/classificador/dup.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# dup.to_csv(filepath,index=True) 

Lista de antibióticos combinados, sem duplicatas



In [5]:
no_double_antib = pd.read_excel('/home/joao/documentos/IC-Joao/classificador/lista-antibioticos-sem-duplicatas.xlsx')

In [22]:
#Conferindo se não há duplicatas
y = 0
for x in no_double_antib['InChIKey'].duplicated():
    if x is True:
        y = y + 1
print(y)


0


In [5]:
no_double_antib.shape

(482, 3)

In [20]:
coconut.shape

(4371, 23)

In [23]:
#Falta verificar manualmente se todas as moléculas são efetivamente antibióticos

Adicionando coluna de 1 para indicar a classe de antibiótico

In [6]:
one = np.ones(482,dtype=int)
no_double_antib['Classe'] = one

Adicionando a coluna de 0 para indicar a classe de não antibiótico


In [21]:
zero = np.zeros(4371, dtype=int)
coconut['Classe'] = zero

Juntando os dados de antibióticos e não antibióticos

In [22]:
coconut.columns

Index(['coconut_id', 'inchi', 'inchikey', 'SMILES', 'sugar_free_smiles',
       'molecular_formula', 'molecular_weight', 'citationDOI', 'textTaxa',
       'name', 'synonyms', 'NPL_score', 'number_of_carbons',
       'number_of_nitrogens', 'number_of_oxygens', 'number_of_rings',
       'total_atom_number', 'bond_count', 'found_in_databases',
       'murko_framework', 'alogp', 'apol', 'topoPSA', 'Classe'],
      dtype='object')

In [29]:
coconut_renamed = coconut.rename(columns = {'name':'Name','inchikey':'InChIKey'})

In [35]:
coconut2 = coconut_renamed[['Name','SMILES','InChIKey','Classe']]

In [37]:
final_dataset = pd.concat([no_double_antib, coconut_renamed[['Name','SMILES','InChIKey','Classe']]])

In [43]:
from pathlib import Path  
filepath = Path('/home/joao/documentos/IC-Joao/classificador/final_data_classifier.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_dataset.to_csv(filepath,index=True)